In [1]:
import cv2
import numpy as np
import os
from mtcnn import MTCNN
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing import image as image_utils

In [2]:
# Load the Model
face_recognition_model = InceptionResNetV2(weights='imagenet', include_top=False)

In [3]:
# Image processing functions
def preprocess_image(image_path):
    img = image_utils.load_img(image_path, target_size=(160, 160))
    img_array = image_utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 127.5 - 1.0
    return img_array

def preprocess_face(face):
    face = cv2.resize(face, (160, 160))
    face = face.astype("float32")
    face = (face - 127.5) / 128
    face = np.expand_dims(face, axis=0)
    return face

In [4]:
# Face detection function
def detect_faces(image, face_recognition_model):
    face_detector = MTCNN()
    face_locations = face_detector.detect_faces(image)
    face_locations = [(face['box'][1], face['box'][0] + face['box'][2], face['box'][1] + face['box'][3], face['box'][0]) for face in face_locations]
    return face_locations

def encode_faces(image, face_locations, face_recognition_model):
    face_encodings = []
    for (top, right, bottom, left) in face_locations:
        face = image[top:bottom, left:right]
        face = preprocess_face(face)
        face_encoding = face_recognition_model.predict(face)[0]
        face_encodings.append(face_encoding)
    return face_encodings

In [5]:
def recognize_faces_in_frame(face_encodings, known_faces, face_recognition_model, tolerance=90):
    recognized_names = []
    recognized_face_locations = []

    # Compare each face encoding with the known face database
    for face_encoding in face_encodings:
        if len(face_encoding) == 0:  
            continue
        distances = {name: np.linalg.norm(encoding - face_encoding) for name, encoding in known_faces.items()}
        min_distance_name = min(distances, key=distances.get)
        min_distance = distances[min_distance_name]

        if min_distance <= tolerance:
            recognized_names.append(min_distance_name)
        else:
            recognized_names.append("Unknown")

    return recognized_names, face_locations 


In [6]:
# Function to create a known faces database
def create_known_faces_database(people_names, people_image_folders, face_recognition_model):
    known_faces = {}
    for i, name in enumerate(people_names):
        print(f"Processing images of {name}")
        images_folder = people_image_folders[i]
        face_encodings = []
        for filename in os.listdir(images_folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # I used PNG
                image_path = os.path.join(images_folder, filename)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                face_locations = detect_faces(image, face_recognition_model)
                for face_location in face_locations:
                    top, right, bottom, left = face_location
                    face = image[top:bottom, left:right]
                    face = preprocess_face(face)
                    face_encoding = face_recognition_model.predict(face)[0]
                    face_encodings.append(face_encoding)
        if face_encodings:
            known_faces[name] = np.mean(face_encodings, axis=0) 
    return known_faces

In [7]:
# Load the InceptionResNetV2 model
face_recognition_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Set people names and image folder paths
people_names = ['Jenny', 'Mithun', 'Ronak', 'Omkarini', 'Sandeep']
people_image_folders = ['/Users/jenny/Documents/Spring 2024/CIS 515/Final/Jenny',
                        '/Users/jenny/Documents/Spring 2024/CIS 515/Final/Mithun',
                        '/Users/jenny/Documents/Spring 2024/CIS 515/Final/Ronak',
                        '/Users/jenny/Documents/Spring 2024/CIS 515/Final/Omkarini',
                        '/Users/jenny/Documents/Spring 2024/CIS 515/Final/Sandeep', ]

# Create a pre-trained face database
known_faces = create_known_faces_database(people_names, people_image_folders, face_recognition_model)


Processing images of Jenny
1/1 [==============================] - 1s 1s/step
Processing images of Mithun
1/1 [==============================] - 0s 82ms/step
Processing images of Ronak
1/1 [==============================] - 0s 93ms/step
Processing images of Omkarini
1/1 [==============================] - 0s 90ms/step
Processing images of Sandeep
1/1 [==============================] - 0s 91ms/step


In [8]:
# Bounding Box
def draw_boxes(image, recognized_names, recognized_face_locations):
    for (top, right, bottom, left), name in zip(recognized_face_locations, recognized_names):
        cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        label = f"{name} ({left}, {top})"
        cv2.putText(image, label, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
    return image

In [9]:
# Webcam
cap = cv2.VideoCapture(0)


In [10]:
# MTCNN 
face_detector = MTCNN()

In [11]:
try:
    while True:
        # Read the Frame
        ret, img = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Using MTCNN to detect faces
        face_locations = detect_faces(img_rgb, face_detector)

        
        if face_locations:
            face_encodings = encode_faces(img_rgb, face_locations, face_recognition_model)
            recognized_names, recognized_face_locations = recognize_faces_in_frame(face_encodings, known_faces, face_recognition_model)
            img_with_boxes = draw_boxes(img_rgb, recognized_names, recognized_face_locations)
        else:
            img_with_boxes = img_rgb

        img_with_boxes = cv2.cvtColor(img_with_boxes, cv2.COLOR_RGB2BGR)
        cv2.imshow('Face Recognition', img_with_boxes)

        # Press ESC to finish
        if cv2.waitKey(1) & 0xFF == 27:
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 10ms/step


KeyboardInterrupt: 